In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()

In [ ]:
# Read the original json file and make a graph representation of the json structure
print('Reading JSON File... ')
_all_ = open('../../../data/kaggle_arXiv/arxiv-metadata-oai-snapshot.json', encoding='utf-8').read()
print('Separating into lines...')
_jsons_ = []
for _line_ in _all_.split('\n'):
    if len(_line_) == 0: continue
    _jsons_.append(json.loads(_line_))
    if len(_jsons_) > 1000: break # short circuit while developing...
print(f'{len(_jsons_)=}')
print('Analyzing json structure...')
_json_repr_ = rt.jsonRepr(_jsons_)
print('Making graph representation...')
df, relates, labels = _json_repr_.starPathGraphDataframe()
len(df), df.columns

In [ ]:
# Display the graph structure
g   = rt.createNetworkXGraph(df, relates)
pos = rt.hyperTreeLayout(g)
rt.linkNode(df, relates, pos, node_labels=labels, draw_labels=True, node_label_max_w=None, w=1280, h=768)

In [ ]:
xform_spec = """
__id__         = '$.id'         | PaperID   | uniq
__journalref__ = '$.journal-ref'| JournalID | uniq
__id__ --- hasTitle     --- '$.title'         | xsd:string | uniq @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- hasAbstract  --- '$.abstract'      | xsd:string | cont @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- hasSubmitter --- '$.submitter'     | xsd:string | ambi @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- hasDOI       --- '$.doi'           | xsd:string | uniq @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- hasLicense   --- '$.license'       | xsd:string | uniq @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- updateDate   --- '$.update_date'   | xsd:date   | date @@@ __journalref__ ^^^ 'kaggle_arXiv'
__id__ --- hasAuthor    --- '$.authors[*][0]' | xsd:string | ambi @@@ __journalref__ ^^^ 'kaggle_arXiv'
"""
ofv = rt.ontologyFrameworkInstance(xform_spec=xform_spec)
ofv.parse(_jsons_[0])
print(len(ofv.df_triples))
ofv.df_triples.sample(10)

In [ ]:
ofv.uid_lu[100028]

In [ ]:
_jsons_[0]